# Predicting House Prices

<img src="https://upload.wikimedia.org/wikipedia/commons/c/cc/Street_scape_in_Boston-Edison.jpg" width=400/>

Today we are going to building a **linear regression** model to predict house prices in Boston.

#### Review: Unsupervised vs Supervised Learning
*   Supervised learning uses existing inputs and outputs to create a mapping function from inputs to outputs
  * Can be used for regression problems, where outputs are continuous, or classification problems, where output is discrete
*   Unsupervised learning finds relationships and clusters in input data

We use the hypothesis function

$$h(x^{i}) = w^{T}x^{i} + b$$

to make an output prediction for each training example, representing the expected price of a house. We use the following loss function to calculate how accurate our prediction is:

$$Loss = L(y^{i} , a^{i}) = \frac{1}{2}(a^{i} - y^{i})^{2}$$

Then, we apply gradient descent to optimize our weights.

#### Some of the things you will learn
* General architecture of a learning algorithm
* initializing parameters
* doing forward propagation
* calculating the cost function
* doing back propagation with gradient descent

#### Instructions
1. You will write your code between 
```python 
### START CODE HERE ###
```
```python 
### END CODE HERE ###
```

2. Do not use any loops (the necessary loop is included for you!)

## Packages
First let's import some libraries
* [scikit-learn](http://scikit-learn.org/stable/) is used to get our dataset
* [numpy](http://www.numpy.org/) is used for scientific computing
* [matplotlib](https://matplotlib.org/) is used for plotting data
* [pandas](https://pandas.pydata.org/) is used for data processing and analysis
* [seaborn](https://seaborn.pydata.org/) is used for statistical data visualization

In [0]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

## Data Visualization

In [0]:
boston = load_boston()
bos = pd.DataFrame(boston.data)
bos.columns = boston.feature_names
bos = bos.drop('B', axis = 1)
bos['PRICE'] = boston.target
pd.options.display.width = 300
print(bos.head())


         CRIM      per capita crime rate by town
         ZN        proportion of residential land zoned for lots over 25,000 sq.ft.
         INDUS     proportion of non-retail business acres per town
         CHAS      Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
         NOX       nitric oxides concentration (parts per 10 million)
         RM        average number of rooms per dwelling
         AGE       proportion of owner-occupied units built prior to 1940
         DIS       weighted distances to five Boston employment centres
         RAD       index of accessibility to radial highways
         TAX       full-value property-tax rate per $10,000
     PTRATIO   pupil-teacher ratio by town
     LSTAT     % lower status of the population
     MEDV      Median value of owner-occupied homes in $1000's

##

In [0]:
# See the general trend in the prices

sns.distplot(bos['PRICE'])

In [0]:
# Plotting simple relations
# Relation between Crime rate and Price:

plt.plot(bos.iloc[::10, :]['PRICE'].sort_values(), bos.iloc[::10, :].sort_values('PRICE')['CRIM']*100)
plt.xlabel('House price in $1000s')
plt.ylabel('Crime Rate')
plt.show()


In [0]:

# Relation between Age of the house and Price
plt.plot(bos.iloc[::20, :]['PRICE'].sort_values(), bos.iloc[::20, :].sort_values('PRICE')['AGE'])
plt.xlabel('House price in $1000s')
plt.ylabel('Age')
plt.title('Age Vs. Price')
plt.show()

In [0]:
# Pairwise Correlation between the features


current_palette = sns.color_palette()
corr=bos.iloc[:,[0,5,6,8,10,12]].corr()

plt.figure(figsize=(15, 15))

sns.heatmap(corr, vmax=.8, linewidths=0.01, square=True,annot=True,cmap='YlGnBu',linecolor="white")
plt.title('Correlation between features');

In [0]:
print(bos.describe())

In [7]:
X = bos.drop('PRICE', axis = 1)
Y = bos['PRICE']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33, random_state = 5)

X_train = np.asarray(X_train).T
X_test = np.asarray(X_test).T
Y_train = np.asarray(Y_train).reshape(1, Y_train.shape[0])
Y_test = np.asarray(Y_test).reshape(1, Y_test.shape[0])

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(12, 339)
(12, 167)
(1, 339)
(1, 167)


## **Architecture of the Learning Algorithm**

Now it's time to get started on the learning algorithm itself! You will build a linear regression algorithm.


**Mathematical algorithm:**

Given $x^{i}$ which is 1 training example of size (12 , 1), 

$$h(x^{i})=z^{i} = w^{T}x^{i} + b$$

$$Loss = L(y^{i}, a^{i})=(a^{i} - y^{i})^{2}$$

Then to calculate the cost over all m training samples,

$$Cost = J = \frac{1}{2m}\sum _{i = 1}^{m} L(y^{i}, a^{i})$$


The superscript $^{T}$ just means transpose. What this algorithm is doing is calculating the model predictions for each training sample, and then finding the average loss over all the samples where loss is a measure of how far off the predictions are from the correct labels.

**Next steps are:**


1.   Define model structure
2.   Initialize parameters
3.   Implement gradient descent: calculate loss, gradient, and update parameters.




### Initializing Parameters

We need a function that inputs a matrix size, and then creates a weight matrix and a bias that are initialized to zero.

Hint: `np.zeros(shape=(x,y))` returns a numpy array of zeros of shape (x,y)

In [0]:
def initialize_with_zeros(dim):
    """
    This function creates a vector of zeros of shape (dim, 1) for w and initializes b to 0.
    
    Argument:
    dim -- size of the w vector we want (or number of parameters in this case)
    
    Returns:
    w -- initialized vector of shape (dim, 1)
    b -- initialized scalar (corresponds to the bias)
    """
    
    ### START CODE HERE ### (≈ 2 line of code)
    w = None
    b = None
    ### END CODE HERE ###

    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w, b

In [0]:
dim = 2
w, b = initialize_with_zeros(dim)
print ("w = " + str(w))
print ("b = " + str(b))

### Forward Propagation

Now it's time to implement forward propagation. During forward propagation we will compute the model predictions, the cost over all of our samples and the gradients for the weights and bias. Here are the steps we will need to take:

1. use the equation $w^{T}X + b$ to compute $A$  *(the capital $X$ means that it is a matrix containing all of our $x^{i}$ samples)*
3. use the cost function described earlier to compute our cost $J$
4. compute the gradients using the equations given below

As a reminder our cost function is $J = \frac{1}{2m}\sum _{i = 1}^{m} (a^{i} - y^{i})^{2}$

**Gradients:**


$$db = \frac{\partial J}{\partial b} = \frac{1}{m}\sum _{i = 1}^{m} a^{i} - y^{i}$$
$$$$
$$dw = \frac{\partial J}{\partial w} = \frac{1}{m}\sum _{i = 1}^{m} (a^{i} - y^{i})x^{i}$$

If you have taken multivariable calculus then these equations are the partial derivatives of our cost function $J$ with respect to our weights $w$ and our bias $b$. By taking their negatives we can optimize our cost function in the direction of steepest descent. If the last two scentences made no sense then fear not. You can simply implement these equations without fully understanding them. Just know that using these equations we can reduce our cost and move closer to a loss of 0 (i.e. a correct prediction).
$$$$

Hint: 
* you can use `np.dot()` to multiply two matrices together
* you can use `np.sum()` to take the sum of all the elements in a matrix

In [0]:
def propagate(w, b, X, Y):
    """
    Implement the cost function and its gradient for the propagation explained above

    Arguments:
    w -- weights, a numpy array of size (13, 1)
    b -- bias, a scalar
    X -- data of size (12, number of examples)
    Y -- house prices (in 1000s)

    Return:
    cost -- mean squared error
    dw -- gradient of the loss with respect to w, thus same shape as w
    db -- gradient of the loss with respect to b, thus same shape as b
    
    Tips:
    - Write your code step by step for the propagation
    """
    
    m = X.shape[1]
    
    # FORWARD PROPAGATION (FROM X TO COST)
    ### START CODE HERE ### (≈ 2 lines of code)
    A = None # compute activation
    cost =  None # compute cost
    ### END CODE HERE ###
    
    # BACKWARD PROPAGATION (TO FIND GRAD)
    ### START CODE HERE ### (≈ 2 lines of code)
    dw = None
    db = None
    ### END CODE HERE ###

    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

In [0]:
w, b, X, Y = np.array([[1], [2]]), 2, np.array([[1,2], [3,4]]), np.array([[1, 0]])
grads, cost = propagate(w, b, X, Y)
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))
print ("cost = " + str(cost))

Expected Output:

* dw = [[16.] [ 36.]]
* db = 10.0
* cost = 52.0

### Backpropagation

Now we want to use the gradients that we computed earlier to perform back propagation. Back propagation is process we use to update our weights bias by a small amount so that our model gets a little better better at making predictions. Here are the equations in case you forgot:

$$w = w - \alpha * dw$$
$$b = b - \alpha * db$$

$\alpha$ is our learning rate, which is often a small number like .01, .001, or .000001. If you remember from the slides, we want to update our weights and bias without overshooting the minimum for our cost function. The picture below illustrates this.

<img src="https://storage.googleapis.com/aibootcamp/Week%201/assets/divergence.jpg" width="350" height="240"/>

In [0]:
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    """
    This function optimizes w and b by running a gradient descent algorithm
    
    Arguments:
    w -- weights, a numpy array of size (12, 1)
    b -- bias, a scalar
    X -- data of shape (12, number of examples)
    Y -- house prices (in 1000s)
    num_iterations -- number of iterations of the optimization loop
    learning_rate -- learning rate of the gradient descent update rule
    print_cost -- True to print the loss every 100 steps
    
    Returns:
    params -- dictionary containing the weights w and bias b
    grads -- dictionary containing the gradients of the weights and bias with respect to the cost function
    costs -- list of all the costs computed during the optimization, this will be used to plot the learning curve.
    
    Tips:
    You basically need to write down two steps and iterate through them:
        1) Calculate the cost and the gradient for the current parameters. Use propagate().
        2) Update the parameters using gradient descent rule for w and b.
    """
    
    costs = []
    
    for i in range(num_iterations):
        
        
        # Cost and gradient calculation (≈ 1 line of code)
        ### START CODE HERE ### 
        grads, cost = None
        ### END CODE HERE ###
        
        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]
        
        # update rule (≈ 2 lines of code)
        ### START CODE HERE ###
        w = None
        b = None
        ### END CODE HERE ###
        
        # Record the costs
        if i != 0 and i % 1000 == 0:
            costs.append(cost)
        
        # Print the cost every 100 training examples
        if print_cost and i != 0 and i % 1000 == 0:
            print ("Cost after iteration %i: %f" % (i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [0]:
params, grads, costs = optimize(w, b, X, Y, num_iterations= 100, learning_rate = 0.009, print_cost = False)

print ("w = " + str(params["w"]))
print ("b = " + str(params["b"]))
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))

Expected Output:
* **w = ** [[-0.08140605] [-0.24334521]]
* **b = ** 1.419030420562572
* **dw = ** [[ 0.08670598] [-0.02296021]]
* **db = ** -0.05483309171519252

### Predictions

Once we have trained our model, we need a way to use it. For that we will create a function called predict that will take an input of training samples, a weight matrix and a bias, and then output predictions price of the house.

$$A = w^{T}X + b$$

In [0]:
def predict(w, b, X):
    '''
    Predict the house price using learned linear regression parameters (w, b)
    
    Arguments:
    w -- weights, a numpy array of size (12, 1)
    b -- bias, a scalar
    X -- data of size (12, number of examples)
    
    Returns:
    Y_prediction -- a numpy array (vector) containing all predictions (0/1) for the examples in X
    '''
    
    m = X.shape[1]
    
    # Compute vector "A" predicting the the house price
    ### START CODE HERE ### (≈ 1 line of code)
    prediction = None
    ### END CODE HERE ###
    
    assert(prediction.shape == (1, m))
    
    return prediction

In [0]:
print("predictions = " + str(predict(w, b, X)))

Expected output:
* [[9 12]]

### Model

Now that we all the pieces of linear regression, lets put all together into one function. 

Implement the model function. Use the following notation:

* `Y_prediction` for your predictions on the test set
* `Y_prediction_train` for your predictions on the train set
* `w`, `costs`, `grads` for the outputs of optimize()

In [0]:
def model(X_train, Y_train, X_test, Y_test, num_iterations=2000, learning_rate=0.01, print_cost=False):
    """
    Builds the linear regression model by calling the function you've implemented previously
    
    Arguments:
    X_train -- training set represented by a numpy array of shape (12, m_train)
    Y_train -- training labels represented by a numpy array (vector) of shape (1, m_train)
    X_test -- test set represented by a numpy array of shape (12, m_test)
    Y_test -- test labels represented by a numpy array (vector) of shape (1, m_test)
    num_iterations -- hyperparameter representing the number of iterations to optimize the parameters
    learning_rate -- hyperparameter representing the learning rate used in the update rule of optimize()
    print_cost -- Set to true to print the cost every 100 iterations
    
    Returns:
    d -- dictionary containing information about the model.
    """
    
    ### START CODE HERE ###
    # initialize parameters with zeros (≈ 1 line of code)
    w, b = None

    # Gradient descent (≈ 1 line of code)
    parameters, grads, costs = None
    
    # Retrieve parameters w and b from dictionary "parameters"
    w = None
    b = None
    
    # Predict test/train set examples (≈ 2 lines of code)
    prediction_test = None
    prediction_train = None

    
    ### END CODE HERE ###

    #convert error to RMSE
    train_rmse = np.sqrt( 1 / X_train.shape[1] * np.sum(( prediction_train - Y_train ) ** 2) )
    test_rmse = np.sqrt( 1 / X_test.shape[1] * np.sum(( prediction_test - Y_test ) ** 2) )
    
    # Print train/test Errors
    print("\ntrain RMSE: {}".format(train_rmse))
    print("test RMSE: {}".format(test_rmse))

    
    d = {"costs": costs,
         "Y_prediction_test": prediction_test, 
         "Y_prediction_train" : prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [0]:
d = model(X_train, Y_train, X_test, Y_test, num_iterations = 500000, learning_rate = 0.00001, print_cost = True)

Expected Output:
* train RMSE: 6.0800950740323705
* test RMSE: 6.491753239651829

In [0]:
costs = np.squeeze(d['costs'])
plt.plot(costs)
plt.ylabel('cost')
plt.xlabel('iterations (per thousands)')
plt.title("Learning rate =" + str(d["learning_rate"]))
plt.show()